In [1]:
import sys
import numpy as np
import pandas as pd

# Compute return rate over a given price vector, with 3 modifiable parameters
def computeReturnRate(priceVec, types, param):
    capital=10000    # Initial available capital
    capitalOrig=capital     # original capital
    dataCount=len(priceVec)                # day size
    suggestedAction=np.zeros((dataCount,1))    # Vec of suggested actions
    stockHolding=np.zeros((dataCount,1))      # Vec of stock holdings
    total=np.zeros((dataCount,1))         # Vec of total asset
    realAction=np.zeros((dataCount,1))    # Real action, which might be different from suggested action. For instance, when the suggested action is 1 (buy) but you don't have any capital, then the real action is 0 (hold, or do nothing). 
    # Run through each day
    for ic in range(dataCount):
        currentPrice=priceVec[ic]    # current price
        suggestedAction[ic]=myStrategy(priceVec[0:ic], currentPrice, types, param)        # Obtain the suggested action
        # get real action by suggested action
        if ic>0:
            stockHolding[ic]=stockHolding[ic-1]    # The stock holding from the previous day
        if suggestedAction[ic]==1:    # Suggested action is "buy"
            if stockHolding[ic]==0:        # "buy" only if you don't have stock holding
                stockHolding[ic]=capital/currentPrice # Buy stock using cash
                capital=0    # Cash
                realAction[ic]=1
        elif suggestedAction[ic]==-1:    # Suggested action is "sell"
            if stockHolding[ic]>0:        # "sell" only if you have stock holding
                capital=stockHolding[ic]*currentPrice # Sell stock to have cash
                stockHolding[ic]=0    # Stocking holding
                realAction[ic]=-1
        elif suggestedAction[ic]==0:    # No action
            realAction[ic]=0
        else:
            assert False
        total[ic]=capital+stockHolding[ic]*currentPrice    # Total asset, including stock holding and cash

    np.savetxt("foo.csv", realAction, delimiter=",")
    returnRate=(total[-1]-capitalOrig)/capitalOrig        # Return rate of this run
    return returnRate

In [2]:
def myStrategy(pastPriceVec, currentPrice, types):
    # Using short RSI and long RSI to decide action
    # If short RSI>param[1] -> sell
    # If short RSI<param[2] -> buy
    # Else if short RSI > long RSI -> buy
    # Else if short RSI < long RSI -> sell
 
    import numpy as np
    action=0        # actions=1(buy), -1(sell), 0(hold), with 0 as the default actions
 
'''
    if(types=='SPY'):
        param=[19, 88, 30, 2532]
    elif(types=='DSI'):
        param=[19, 86, 30, 2351]
    elif(types=='IAU'):
        param=[6, 92, 10, 59]
    elif(types=='LQD'):
        param=[17, 94, 26, 80]
    windowSize = param[0]
'''
 
    dataLen=len(pastPriceVec)        # Length of the data vector
    if dataLen<=param[0]:
        return action
    longrsi=None
 
    U = pastPriceVec[-windowSize:] - pastPriceVec[-windowSize-1:-1]  # Up
    D = -U  # Down
    for i in range(0, windowSize):
        if U[i] < 0:
            U[i] = 0
        else:
            D[i] = 0
    sumU = np.sum(U)
    sumD = np.sum(D)
    rsi = (sumU/(sumU+sumD))*100
 
    U=[]
    D=[]
    if(param[3]<len(pastPriceVec)):
        U = pastPriceVec[-param[3]:] - pastPriceVec[-param[3]-1:-1]  # Up
        D = -U  # Down
    else:
        U = pastPriceVec[1:len(pastPriceVec)] - pastPriceVec[0:len(pastPriceVec)-1]  # Up
        D = -U
 
    for i in range(0, len(U)):
        if U[i] < 0:
            U[i] = 0
        else:
            D[i] = 0
    sumU = np.sum(U)
    sumD = np.sum(D)
    longrsi = (sumU/(sumU+sumD))*100
 
 
    if(rsi>param[1]):
        action=-1
    elif(rsi<param[2]):
        action=1
    elif(longrsi):
        if(rsi>longrsi):
            action=1
        elif(rsi<longrsi):
            action=-1  
    else:
        windowSize=param[4]
        alpha=param[5]
        beta=param[6]
        action=0    # action=1(buy), -1(sell), 0(hold), with 0 as the default action
        dataLen=len(pastPriceVec)        # Length of the data vector
        # Compute MA
        if dataLen<windowSize:
            ma=np.mean(pastPriceVec)    # If given price vector is small than windowSize, compute MA by taking the average
        else:
            windowedData=pastPriceVec[-windowSize:]        # Compute the normal MA using windowSize 
            ma=np.mean(windowedData)
        # Determine action
        if (currentPrice-ma)>alpha:        # If price-ma > alpha ==> buy
            action=1
        elif (currentPrice-ma)<-beta:    # If price-ma < -beta ==> sell
            action=-1
    return action

In [19]:
from copy import deepcopy
import multiprocessing as mp
import itertools
from tqdm import tqdm_notebook as tqdm
if __name__=='__main__':
    returnRateBest=-1.00     # Initial best return rate
    file_name = "data/DSI.csv"
    df=pd.read_csv(file_name)    # read stock file
    adjClose=df["Adj Close"].values        # get adj close as the price vector
    param = [0,0,0]
    params = list(itertools.product(range(17,23), range(84,89), range(27,32), range(2349,2352), range(15,20), range(0,11), range(4,11)))

    
    for param in tqdm(params):
        returnRate = computeReturnRate(adjClose, "SPY", param) 
        if returnRate > returnRateBest:        # Keep the best parameters
            param_best = deepcopy(param)
            returnRateBest = returnRate
            print(param_best, returnRateBest)

    print("Best settings: {} ==> returnRate={}".format(param_best, returnRateBest))   

    
    # Print the best result
    # DSI: (19, 86, 30, 2351) [3.44115803]

(17, 84, 27, 2349, 15, 0, 4) [0.51217713]
(17, 84, 27, 2349, 16, 0, 4) [0.51955983]
(17, 84, 28, 2349, 15, 0, 4) [0.67101827]
(17, 84, 28, 2349, 16, 0, 4) [0.67917645]
(17, 84, 29, 2349, 15, 0, 4) [0.83539492]
(17, 84, 29, 2349, 16, 0, 4) [0.84435561]
(17, 84, 31, 2349, 15, 0, 4) [0.87476304]
(17, 84, 31, 2349, 16, 0, 4) [0.88438939]
(17, 86, 29, 2349, 15, 0, 4) [0.98519442]
(17, 86, 29, 2349, 16, 0, 4) [0.99488646]
(17, 86, 31, 2349, 15, 0, 4) [1.02777565]
(17, 86, 31, 2349, 16, 0, 4) [1.03818768]
(17, 88, 29, 2349, 15, 0, 4) [1.10709827]
(17, 88, 29, 2349, 16, 0, 4) [1.11738546]
(17, 88, 31, 2349, 15, 0, 4) [1.12395738]
(17, 88, 31, 2349, 16, 0, 4) [1.13486327]
(18, 84, 31, 2349, 15, 0, 4) [1.21540606]
(18, 86, 31, 2349, 15, 0, 4) [1.30306048]
(18, 88, 31, 2349, 15, 0, 4) [1.37738965]
(19, 84, 30, 2349, 15, 0, 4) [1.48659955]
(19, 84, 30, 2349, 17, 0, 4) [1.50685738]
(19, 84, 30, 2349, 18, 0, 4) [1.51507987]
(19, 84, 30, 2349, 19, 0, 4) [1.55162505]
(19, 84, 31, 2349, 19, 0, 4) [1.55

In [ ]:
from copy import deepcopy
import multiprocessing as mp
import itertools
from tqdm import tqdm_notebook as tqdm
if __name__=='__main__':
    returnRateBest=-1.00     # Initial best return rate
    file_name = "data/DSI.csv"
    df=pd.read_csv(file_name)    # read stock file
    adjClose=df["Adj Close"].values        # get adj close as the price vector
    param = [0,0,0]
    params = list(itertools.product(list(range(2, 20)), list(range(70, 101, 2)), list(range(0, 31, 2))))

    print(mp.cpu_count())
    
    for param in tqdm(params):
        returnRate = computeReturnRate(adjClose, "SPY", param) 
        if returnRate > returnRateBest:        # Keep the best parameters
            param_best = deepcopy(param)
            returnRateBest = returnRate
            print(param_best, returnRateBest)

    print("Best settings: {} ==> returnRate={}".format(param_best, returnRateBest))   
    #pool.close()   

    
    # Print the best result
    

In [ ]:
from copy import deepcopy
import multiprocessing as mp
import itertools
from tqdm import tqdm_notebook as tqdm
if __name__=='__main__':
    returnRateBest=-1.00     # Initial best return rate
    file_name = "data/IAU.csv"
    df=pd.read_csv(file_name)    # read stock file
    adjClose=df["Adj Close"].values        # get adj close as the price vector
    param = [0,0,0]
    params = list(itertools.product(list(range(50, 100)), list(range(70, 100)), list(range(0, 30))))

    print(mp.cpu_count())
    
    for param in tqdm(params):
        returnRate = computeReturnRate(adjClose, "SPY", param) 
        if returnRate > returnRateBest:        # Keep the best parameters
            param_best = deepcopy(param)
            returnRateBest = returnRate
            print(param_best, returnRateBest)

    print("Best settings: {} ==> returnRate={}".format(param_best, returnRateBest))   
    #pool.close()   

    
    # Print the best result
    

In [23]:
if __name__=='__main__':
    fileList=['data/SPY.csv', 'data/DSI.csv', 'data/IAU.csv', 'data/LQD.csv']
    fileCount=len(fileList);
    rr=np.zeros((fileCount,1))
    for ic in range(fileCount):
        file=fileList[ic];
        df=pd.read_csv(file)    
        adjClose=df["Adj Close"].values    # Get adj close as the price vector
        stockType=file[-7:-4]        # Get stock type
        rr[ic]=computeReturnRate(adjClose, stockType, param)    # Compute return rate
        print("file=%s ==> rr=%f" %(file, rr[ic]));
    print("Average return rate = %f" %(np.mean(rr)))

file=data/SPY.csv ==> rr=4.269673
file=data/DSI.csv ==> rr=3.441158
file=data/IAU.csv ==> rr=5.061652
file=data/LQD.csv ==> rr=1.682696
Average return rate = 3.613795


In [ ]:
# MA : 2.575183
#   SPY: windowSize=40, alpha=7, beta=20 ==> returnRate=3.193493
#   DSI: windowSize=17, alpha=2, beta=10 ==> returnRate=3.282897
#   IAU: windowSize=26, alpha=0, beta=2 ==> returnRate=2.512694
#   LQD: windowSize=5, alpha=0, beta=1 ==> returnRate=1.332634

In [ ]:
# EMA : 
#   SPY: windowSize=37, alpha=7, beta=20 ==> returnRate=3.227214
#   DSI: windowSize=17, alpha=2, beta=10 ==> returnRate=3.282897
#   IAU: windowSize=26, alpha=0, beta=2 ==> returnRate=2.358789
#   LQD: windowSize=5, alpha=0, beta=1 ==> returnRate=1.332634

In [ ]:
# RSI : 2.520975
#   SPY: (19, 88, 30) ==> returnRate=[4.08856354]
#   DSI: (19, 86, 30) ==> returnRate=[3.33668019]
#   IAU: (6, 92, 10) ==> returnRate=[4.72002328]
#   LQD: (17, 94, 26) ==> returnRate=[1.26393499]

# SPY: (19, 88, 30, 2532) [4.26967304]
# DSI: (19, 86, 30, 2351) [3.44115803]
# IAU: (6, 92, 10, 59) [5.27399206]
# LQD: (17, 94, 26, 80) [1.68269565]

In [ ]:
# MACD :
#   SPY: windowSize=8, alpha=1.0, beta=-1.9 ==> returnRate=2.741027

In [ ]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

df = pd.read_csv('data/SPY.csv')

fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.show()